In [ ]:
# import the necessery libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense,Activation,Lambda,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K

In [ ]:
''' use this code if you want to run this in google collab
from google.colab import drive
drive.mount('/content/drive')'''

In [ ]:
# defining function for loading the training data and loading the data generator
def load_dataset(csv_path):
    df = pd.read_csv(csv_path)
    labels = df['label'].values
    images = df.drop('label', axis=1).values.reshape(-1, 28, 28, 1)  
    images = images / 255.0  
    return images, labels
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest'
)

In [ ]:
# loading the data
csv_path='train.csv'
x_train, y_train = load_dataset(csv_path)

In [ ]:
# spliting the train and dev
x_train, x_dev, y_train, y_dev = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# reading, reshaping and normalizing the test data
x_test = pd.read_csv('test.csv')
x_test = x_test.values.reshape(-1, 28, 28, 1)
x_test = x_test/255.0

In [ ]:
# one-hot encoding the data
y_train = to_categorical(y_train)
y_dev = to_categorical(y_dev)

In [ ]:
# fit the data to datagenerator
datagen.fit(x_train)
# Generate augmented data batches
augmented_data = datagen.flow(x_train, y_train, batch_size=128)

In [ ]:
...
# define the mish function
def mish(x):
    return x * K.tanh(K.softplus(x))
# define the model
def cnn_model():
 model = Sequential()
 model.add(Conv2D(16, (5, 5), input_shape=(28, 28, 1)))
 model.add(Lambda(mish))
 model.add(MaxPooling2D(pool_size=(2, 2)))
 model.add(Conv2D(32, (3, 3)))
 model.add(Lambda(mish))
 model.add(MaxPooling2D(pool_size=(2, 2)))
 model.add(Dropout(0.1))
 model.add(Flatten())
 model.add(Dense(128, activation='relu'))
 model.add(Dense(50, activation='relu'))
 model.add(Dense(10, activation='softmax'))
 # Compile model
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [ ]:
# define early stopping monitor
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
# build the model
model = cnn_model()

# Lists to store training history
train_loss = []
val_loss = []
train_accuracy = []
val_accuracy = []

# Fit the model
history = model.fit(augmented_data, validation_data=(x_dev, y_dev), epochs=100, batch_size=128, callbacks=[early_stopping])

# Retrieve training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot the training and validation accuracy
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()


In [ ]:
# Predict labels for the test set
predictions = model.predict(x_test)
predicted_labels = np.argmax(predictions, axis=1)

# Create a DataFrame with ImageId and Label columns
results = pd.DataFrame({'ImageId': np.arange(1, len(predicted_labels) + 1), 'Label': predicted_labels})

# export the data
results.to_csv('predictions.csv', index=False)